In [1]:
# YOUR IMPORTS HERE
import matplotlib.pyplot as plt # for plotting
import numpy as np # for transformation

import torch # PyTorch package
import torchvision # load datasets
import torchvision.transforms as transforms # transform data
import torch.nn as nn # basic building block for neural neteorks
import torch.nn.functional as F # import convolution functions like Relu
import torch.optim as optim # optimzer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [2]:
train_data = np.load('/kaggle/input/mlprojectdata/train_images.npy')
train_labels = np.load('/kaggle/input/mlprojectdata/train_labels.npy')
val_data = np.load('/kaggle/input/mlprojectdata/val_images.npy')
val_labels = np.load('/kaggle/input/mlprojectdata/val_labels.npy')
test_data = np.load('/kaggle/input/mlprojectdata/test_images.npy')
test_labels = np.load('/kaggle/input/mlprojectdata/test_labels.npy')

In [3]:
# custom dataset object for dataloaders

class My_Dataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], int(self.y_data[index][0])
        
    def __len__ (self):
        return len(self.X_data)

train_data_for_loader = My_Dataset(torch.FloatTensor(train_data), train_labels)
val_data_for_loader = My_Dataset(torch.FloatTensor(val_data), val_labels)
test_data_for_loader = My_Dataset(torch.FloatTensor(test_data), test_labels)

train_loader = DataLoader(train_data_for_loader, batch_size = 100, shuffle = True)
val_loader = DataLoader(val_data_for_loader, batch_size = len(val_data_for_loader), shuffle = False)
test_loader = DataLoader(test_data_for_loader, batch_size = len(test_data_for_loader), shuffle = False)

In [4]:
# define and instantiate model

class Model(nn.Module):
    # YOUR CODE HERE
    def __init__(self):
        super().__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64 ,4)

    def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = torch.flatten(x, 1)# flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = Model()

In [5]:
# define and instantiate loss function & optimizer

# YOUR CODE HERE
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# train model

# YOUR CODE HERE

# keeping-track-of-losses 
train_losses = []
valid_losses = []

for epoch in range(1, 11):
    # keep-track-of-training-and-validation-loss
    train_loss = 0.0
    valid_loss = 0.0
    
    
    for data, target in train_loader:
        # move-tensors-to-GPU 
        
        # clear-the-gradients-of-all-optimized-variables
        optimizer.zero_grad()
        # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
        output = model(data.view(-1,784))
        # calculate-the-batch-loss
        loss = criterion(output, target)
        # backward-pass: compute-gradient-of-the-loss-wrt-model-parameters
        loss.backward()
        # perform-a-ingle-optimization-step (parameter-update)
        optimizer.step()
        # update-training-loss
        train_loss += loss.item() * data.size(0)
        
    # validate-the-model
    model.eval()
    for data, target in val_loader:
        
        
        output = model(data.view(-1,784))
        
        loss = criterion(output, target)
        
        # update-average-validation-loss 
        valid_loss += loss.item() * data.size(0)
    
    # calculate-average-losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(val_loader.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
        
    # print-training/validation-statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

Epoch: 1 	Training Loss: 0.977476 	Validation Loss: 0.866815
Epoch: 2 	Training Loss: 0.833402 	Validation Loss: 0.774247
Epoch: 3 	Training Loss: 0.764661 	Validation Loss: 0.734064
Epoch: 4 	Training Loss: 0.730930 	Validation Loss: 0.716077
Epoch: 5 	Training Loss: 0.705355 	Validation Loss: 0.664735
Epoch: 6 	Training Loss: 0.686452 	Validation Loss: 0.657654
Epoch: 7 	Training Loss: 0.673567 	Validation Loss: 0.682813
Epoch: 8 	Training Loss: 0.663450 	Validation Loss: 0.660161
Epoch: 9 	Training Loss: 0.654658 	Validation Loss: 0.669478
Epoch: 10 	Training Loss: 0.649528 	Validation Loss: 0.674620


In [7]:
# check accuracy on entire test set

# count correct predictions
with torch.no_grad():
    correct = 0
    for X_test, y_test in test_loader:
        y_pred = model(X_test.view(len(X_test), -1))
        predicted = torch.max(y_pred, 1)[1]
        correct += (predicted == y_test).sum()

# calculate accuracy, print
print(f' Accuracy: {correct.item() / len(test_data_for_loader)}')

 Accuracy: 0.506


In [8]:
# FUNCTION TO SAVE PREDICTIONS TO CSV
# call with trained model and DataLoader for test data
# will save predictions to .csv in current directory
def save_predictions(M, T):
    with open('submission.csv', 'w') as out_file:
        s = 'Id,Category\n'
        with torch.no_grad():
            for X_test, y_test in T:
                y_pred = model(X_test.view(len(X_test), -1))
                predicted = torch.max(y_pred, 1)[1]
                for i in range(len(predicted)):
                    s += f'{i},{str(int(predicted[i]))}\n'
        s = s[:-1]
        out_file.write(s)
        
save_predictions(model, test_loader)